# 3D descriptors experiment

In [1]:
from rdkit.Chem import PandasTools

frame = PandasTools.LoadSDF('sweet.sdf',smilesName='SMILES',molColName='Molecule',includeFingerprints=True)



RDKit WARNING: [14:14:05] Warning: molecule is tagged as 3D, but all Z coords are zero


In [4]:
import numpy as np

def checkAtomsCoordinates(m):
    '''
        Function to check if a molecule contains zero coordinates in all atoms. 
        Then this molecule must be eliminated.
        Returns True if molecules is OK and False if molecule contains zero coordinates.
        Example:
            # Load  test set to a frame
            sdf = 'miniset.sdf'
            df = pt.LoadSDF(sdf, molColName='mol3DProt')
            ## Checking if molecule contains only ZERO coordinates,
           ##  then remove that molecules from dataset
            df['check_coordinates'] = [checkAtomsCoordinates(x) for x in df.mol3DProt]
            df_eliminated_mols = dfl[df.check_coordinates == False]
            df = df[df.check_coordinates == True]
            df.drop(columns=['check_coordinates'], inplace=True)
            print('final minitest set:', df.shape[0])
            print('minitest eliminated:', df_eliminated_mols.shape[0])
    '''
    conf = m.GetConformer()
    position = []
    for i in range(conf.GetNumAtoms()):
        pos = conf.GetAtomPosition(i)
        position.append([pos.x, pos.y, pos.z])
    position = np.array(position)
    if not np.any(position):
        return(False)
    else:
        return(True)

In [19]:
from rdkit.Chem import SDMolSupplier
from rdkit.Chem import rdMolDescriptors as molDesc
from mordred import Calculator, descriptors


def load_sdf_file(file):

    supplier = SDMolSupplier(file)
    mols, attempts = [], 0

    while not mols and attempts < 10:
        mols = list(supplier)
        attempts += 1
    print(f"Loaded {len(mols)} molecules after {attempts} attempts.")

    return mols

def generate_descriptor_rdkit(mols):
    
    descript = []
    ids = []
    y = []

    for mol in mols:
        
        viable = checkAtomsCoordinates(mol)
        
        
        if viable:
            
            y.append(float(mol.GetProp("_SWEET")))
            db_id = int(mol.GetProp("_SourceID"))
            desPBF = molDesc.CalcPBF(mol)
            desRDF = molDesc.CalcRDF(mol)
            desRG = molDesc.CalcRadiusOfGyration(mol)
            desAUTOCORR3D = molDesc.CalcAUTOCORR3D(mol)
            desc_inertial_shape_factor = molDesc.CalcInertialShapeFactor(mol)
            desc_eccentricity = molDesc.CalcEccentricity(mol)
            desc_asphericity = molDesc.CalcAsphericity(mol)
            desc_spherocity_index = molDesc.CalcSpherocityIndex(mol)
            desc_morse = molDesc.CalcMORSE(mol)
            desc_whim = molDesc.CalcWHIM(mol)
            desc_getaway = molDesc.CalcGETAWAY(mol)
            #fp = molDesc.GetHashedAtomPairFingerprintAsBitVect(mol,
             #                                                           nBits = 1024,
              #                                                          includeChirality = True)
            #fp = list(np.asarray(fp, dtype=np.float))
            
            des1 = desRDF + [desPBF, desRG, desc_inertial_shape_factor, desc_eccentricity, desc_asphericity, desc_spherocity_index]+ desAUTOCORR3D + desc_morse + desc_whim 
            
            descript.append(des1)
            ids.append(db_id)
    
    return ids,descript,y

def generate_descriptor_mordred(mols):
    
    descript = []
    ids = []
    y = []
    
    i = 0
    for mol in mols:
        
        i += 1
        viable = checkAtomsCoordinates(mol)
        
        
        if i%100 == 0:
            
            print(i)
        
        if viable:
            
            db_id = int(mol.GetProp("_SourceID"))
            y.append(float(mol.GetProp("_SWEET")))
            calculator = Calculator(descriptors,ignore_3D=False)
            err = calculator(mol)
            desc = err.drop_missing()
            descript.append(list(desc))
            ids.append(db_id)
    
    return ids,descript,y

from padelpy.functions import from_mdl

def featurize_padel(file):
    
    fingerprints = from_mdl(file)
    
    return fingerprints
       

In [2]:
mols_not_sweet = load_sdf_file("not_sweet.sdf")

Loaded 10 molecules after 1 attempts.


In [2]:
#mols_not_sweet = load_sdf_file("not_sweet_2.sdf")

mols_sweet = load_sdf_file("./data/dataset_sweet_3D.sdf")



RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [20:42:25] Warning: molecule

Loaded 25795 molecules after 1 attempts.


RDKit WARNING: [20:42:37] Warning: molecule is tagged as 3D, but all Z coords are zero


In [85]:
from mordred import Calculator, descriptors, get_descriptors_in_module, CPSA,GeometricalIndex,GravitationalIndex,MoRSE, MomentOfInertia
from mordred.CPSA import PNSA, PPSA, DPSA
from mordred.GeometricalIndex import Radius3D

new_descriptors = get_descriptors_in_module( MomentOfInertia)

calculator = Calculator( descriptors,ignore_3D=False)
err = calculator(mols_sweet[300])
desc = err.drop_missing()
len(desc)
#print(err.error)

1507

In [20]:
#ids_not_sweet,descript_not_sweet,y_not_sweet = generate_descriptor_rdkit(mols_not_sweet)

ids_sweet,descript_sweet,y_sweet = generate_descriptor_rdkit(mols_sweet)

len(descript_sweet[0])


634

In [21]:
print(len(descript_sweet))
print(len(ids_sweet))
print(len(y_sweet))

25786
25786
25786


In [22]:
import pandas as pd

#ids = ids_not_sweet + ids_sweet
#descript = descript_not_sweet + descript_sweet
#y = y_not_sweet + y_sweet

ids = ids_sweet
descript = descript_sweet
y = y_sweet


########################### remove nan ################################

descript = np.array(descript)
ids = np.array(ids)
y = np.array(y)

boo = np.isnan(descript).any(axis=1)
#boo = np.where(descript==0)
indexes_to_remove = [index[0] for index in np.argwhere(boo==True)]

descript = np.delete(descript, indexes_to_remove,axis=0)
ids = np.delete(ids, indexes_to_remove)
y = np.delete(y, indexes_to_remove)


df = pd.DataFrame({"ids":ids, "3D": list(descript), "sweet": y})
df["3D"].shape

(25779,)

In [23]:
descript = np.array(descript)
print(descript.shape)
descript = descript[~np.isnan(descript).any(axis=1), :]
print(descript.shape)

(25779, 634)
(25779, 634)


In [24]:
df = df.drop_duplicates(subset= df.columns.difference(["3D","sweet"]))

In [25]:
from sklearn.model_selection import train_test_split,cross_val_score

not_sweet_to_join = df[df["sweet"]==0].sample(n=1900, random_state=1)
sweet = df[df["sweet"]==1]

final_dataset = not_sweet_to_join.append(sweet)
final_dataset["sweet"] = final_dataset["sweet"].map({1:1,2:0})


X_train,X_test,y_train,y_test = train_test_split(final_dataset["3D"],final_dataset["sweet"],test_size=0.20,random_state=1)

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train, test_size=0.25,random_state=2)


In [26]:
X_train = np.array(list(X_train))
X_test = np.array(list(X_test))
y_train = np.array(y_train)
y_test = np.array(y_test)

X_val = np.array(list(X_val))
y_val = np.array(y_val)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(2265, 634)
(756, 634)
(755, 634)


In [27]:
np.count_nonzero(np.isnan(X_train))

0

In [10]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import pickle

def runRF(data, max_depth, max_features, min_samples_split, bootstrap, criterion, n_estimators):
    print('=== RANDOM FOREST ===')
    X_train, X_test, y_train, y_test = data[0], data[1], data[2], data[3]
    
    rf = RandomForestClassifier(max_depth = max_depth, max_features = max_features, 
                                min_samples_split = min_samples_split, bootstrap = bootstrap, 
                                criterion = criterion, n_estimators = n_estimators, n_jobs = -1)
    
    rf = rf.fit(X_train,y_train)
    
    scores = cross_val_score(rf, X_train, y_train, cv=5, n_jobs = -1)
    print(scores)
    print("CV accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    # save the model to disk
    filename = 'models3D/modelRF.sav'
    pickle.dump(rf, open(filename, 'wb'))
    
    rfc_predict = rf.predict(X_test)
    
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y_test, rfc_predict))
    print('\n')
    print("=== Classification Report ===")
    print(classification_report(y_test, rfc_predict))
    print('\n')
    print("=== Test accuracy ===")
    print(accuracy_score(y_test, rfc_predict))
    print('\n')

    return rf

In [30]:
from sklearn import svm

def runSVM(data, kernel, c, gamma):
    print('=== SVM ===')
    X_train, X_test, y_train, y_test = data[0], data[1], data[2], data[3]
    
    svmc = svm.SVC(kernel=kernel, C=c, gamma = gamma, probability = True).fit(X_train,y_train)
    #svmc = svm.SVC(kernel=kernel, C=c, gamma = gamma).fit(X_train,y_train)
    
    scores = cross_val_score(svmc, X_train, y_train, cv=5, n_jobs = -1)
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    #with kernel='rbf', C=1000, gamma = 0.001  Accuracy: 0.88 (+/- 0.01)
    
    # save the model to disk
    filename = 'models3D/modelSVM.sav'
    pickle.dump(svmc, open(filename, 'wb'))
    
    svmc_predict = svmc.predict(X_test)
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y_test, svmc_predict))
    print('\n')
    print("=== Classification Report ===")
    print(classification_report(y_test, svmc_predict))
    print('\n')
    print("=== Test accuracy ===")
    print(accuracy_score(y_test, svmc_predict))
    print('\n')
    
    return svmc

In [28]:
import tensorflow as tf
from tensorflow.python.keras.backend import set_session

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt


config_tf = tf.compat.v1.ConfigProto()
config_tf.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config_tf)
set_session(sess)

def runDNN(data, optimizer, dropout_rate, hidden_layers, l1, l2,units1,units2):
    def create_model(units1 = 512, units2 = 512, dropout_rate=0.0, hidden_layers=1, l1 = 0, l2 = 0, 
                        optimizer = 'adam', batchNormalization = True):
        # create model
        print("unit1: ", units1)
        print('dropout_rate = ', dropout_rate)
        model = Sequential()
        model.add(Dense(units=units1, activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

        for i in range(hidden_layers):
            model.add(Dense(units=units2, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
            model.add(BatchNormalization())
            model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        ##Compile model and make it ready for optimization
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        # Reduce lr callback
        
        return model
    
    print('=== DNN ===')
    model = KerasClassifier(build_fn=create_model, dropout_rate = dropout_rate, 
                            hidden_layers = hidden_layers, epochs=500,
                            l1 = l1, l2 = l2, units1 = units1,units2 = units2,verbose=1)

    
    X_train, X_test, y_train, y_test = data[0], data[1], data[2], data[3]
    
    # simple early stopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    # reduce learning rate
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=50, min_lr=0.00001, verbose=0)
    
    #callbacks = [es, reduce_lr]
    callbacks = [reduce_lr]
    #Training
    history = model.fit(X_train, y_train, batch_size=10, callbacks=callbacks, 
                        validation_data = (X_test, y_test))

    print('Training Accuracy: ', np.mean(history.history['accuracy']))
    print('Validation Accuracy: ', np.mean(history.history['val_accuracy']))
    
    dnn_predict = model.predict(X_test)
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y_test, dnn_predict))
    print("=== Classification Report ===")
    print(classification_report(y_test, dnn_predict))
    print('\n')
    print("=== Test accuracy ===")
    print(accuracy_score(y_test, dnn_predict))
    print('\n')
    
    print(model.model.summary())
    
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    model.model.save('models3D/modelDNN.h5')
    
    return model

In [29]:


data = X_train, X_val, y_train , y_val



units = int(len(data[1]))



#model = runRF(data, 10, 'sqrt', 2, False, 'gini', 900) # 0.87
#runRF(data, None, 'sqrt', 11, False, 'gini', 100) #0.87
#runRF(data, 50, 'auto', 8, True, 'gini', 400)
#model = runSVM(data, 'rbf', 10, 0.001)
#svm = runSVM(data, 'rbf', 1000, 0.001)
model = runDNN(data,'Adam', 0.3, 1, 0.001, 0.01,units,units*2)



=== DNN ===
unit1:  755
dropout_rate =  0.3
Epoch 1/500
227/227 [==============================] - 50s 22ms/step - loss: 20.9205 - accuracy: 0.6481 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/500
223/227 [============================>.] - ETA: 0s - loss: 3.4415 - accuracy: 0.6645

KeyboardInterrupt: 

In [159]:
predictions = model.predict(X_test)
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, predictions))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, predictions))
print('\n')
print("=== Test accuracy ===")
print(accuracy_score(y_test, predictions))
print('\n')

23/23 [==============================] - 0s 2ms/step
=== Confusion Matrix ===
[[323  65]
 [ 27 302]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.83      0.88       388
           1       0.82      0.92      0.87       329

    accuracy                           0.87       717
   macro avg       0.87      0.88      0.87       717
weighted avg       0.88      0.87      0.87       717



=== Test accuracy ===
0.8716875871687587




/home/jcapela/miniconda3/envs/deepmol/lib/python3.9/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
